In [41]:
#!/usr/bin/python
from __future__ import division

import numpy as np
import numpy.matlib
import scipy as sp
import wifi_data
import os
from sklearn.naive_bayes import GaussianNB, MultinomialNB
import sklearn.metrics
from sklearn import mixture
from sklearn.manifold import TSNE
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, merge, Merge, Conv1D
from keras.optimizers import SGD
from keras.utils.visualize_util import plot
from keras.utils.np_utils import to_categorical
from keras.callbacks import Callback, TensorBoard

In [2]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [4]:
%matplotlib

Using matplotlib backend: TkAgg


In [5]:
def naiveBayes(x_train, y_train, x_test, y_test):
    x_train = np.abs(x_train)
    x_test = np.abs(x_test)
    mnb  = MultinomialNB()
    mdl = mnb.fit(x_train, y_train)
    y_pred = mdl.predict(x_test)
    prob = mdl.predict_proba(x_test)
    return (y_test == y_pred).sum()/y_test.size, prob

In [6]:
def localization_error(y, y_hat):
    return np.sqrt(np.sum(np.power((y-y_hat),2), axis=1))

def cdf(error, nBins):
    countsError, binsError = np.histogram(error, nBins);
    f = np.cumsum(countsError) / np.sum(countsError);
    return binsError, countsError, f

In [10]:
fname_dataset = "hancock_data.mat"
fname_model = "grid_classifier.h5"
folderLocation = os.path.dirname("/home/murat/wifi_localization/deep-rf-regression/")
# Create the dataset
dataset = wifi_data.wifi_data(folder_location=folderLocation, filename=fname_dataset, normalize=False, missingValues=0, nTraining=1200, nTesting=343, nValidation=0, verbose=False)
# Show the dataset properties
print "WiFi Set: ", dataset.wifi_train.shape, dataset.wifi_test.shape
print "BT Set: ", dataset.bt_train.shape, dataset.bt_test.shape
print "LoRa Set: ", dataset.lora_train.shape, dataset.lora_test.shape

print "Grid Labels (by XY): ", dataset.grid_labels_train.shape, dataset.grid_labels_test.shape
print "Grid Labels (by #): ", dataset.grid_numbers_train.shape, dataset.grid_numbers_test.shape

# Validation split can be omitted, because Keras can create a validation
# split from the training set
if dataset.wifi_valid != None:
    print "Validation Shapes: \n"
    print "\tWiFi: ", dataset.wifi_valid.shape
    print "\tBT: ", dataset.bt_valid.shape
    print "\tLoRa: ", dataset.lora_valid.shape
    print "\tGrid (by xy-): ", dataset.grid_labels_valid.shape
    print "\tGrid (by #): ", dataset.grid_numbers_valid.shape


grid_numbers_train_categorical = to_categorical(dataset.grid_numbers_train, 175)
grid_numbers_test_categorical = to_categorical(dataset.grid_numbers_test, 175)

WiFi Set:  (1200, 8) (343, 8)
BT Set:  (1200, 8) (343, 8)
LoRa Set:  (1200, 8) (343, 8)
Grid Labels (by XY):  (1200, 2) (343, 2)
Grid Labels (by #):  (1200, 1) (343, 1)


In [11]:
accxy, _ = naiveBayes(dataset.lora_train,dataset.grid_numbers_train.ravel(),dataset.lora_test, dataset.grid_numbers_test.ravel())
accx, probx = naiveBayes(dataset.lora_train, dataset.grid_labels_train[:,0].ravel(), dataset.lora_test, dataset.grid_labels_test[:,0].ravel())
accy, proby = naiveBayes(dataset.lora_train, dataset.grid_labels_train[:,1].ravel(), dataset.lora_test, dataset.grid_labels_test[:,1].ravel())


/usr/lib/python2.7/dist-packages/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:
/usr/lib/python2.7/dist-packages/scipy/sparse/coo.py:200: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.row) != 1 or np.rank(self.col) != 1:


In [13]:
print accxy, accx, accy

0.0320699708455 0.30612244898 0.0874635568513


In [35]:
s = 0
for i in range(0,343):
    px = probx[i,:].reshape(1,probx[i,:].size)
    py = proby[i,:].reshape(proby[i,:].size,1)
    #print px
    #print py
    print dataset.grid_labels_test[i,:]
    #print "px shape: ", px.shape, "py shape", py.shape
    belief_x = np.matlib.repmat(px, 25,1)
    belief_y = np.matlib.repmat(py, 1,7)
    # print belief_x.shape, belief_y.shape
    belief = np.multiply(belief_x, belief_y)
    belief = belief / np.sum(belief)
    print "gt: ", dataset.grid_labels_test[i,:], "x: ", np.argmax(px), "y: ", np.argmax(py)
    qq = dataset.grid_labels_test[i,:] == np.hstack((np.argmax(px), np.argmax(py))) 
    ss = np.logical_and(qq[0], qq[1])
    #print qq, ss
    if(ss):
        s+=1
s/343

[ 0 23]
gt:  [ 0 23] x:  0 y:  18
[ 1 18]
gt:  [ 1 18] x:  6 y:  11
[0 2]
gt:  [0 2] x:  6 y:  19
[0 3]
gt:  [0 3] x:  6 y:  1
[ 5 17]
gt:  [ 5 17] x:  5 y:  11
[ 6 21]
gt:  [ 6 21] x:  0 y:  18
[ 3 17]
gt:  [ 3 17] x:  5 y:  19
[4 7]
gt:  [4 7] x:  5 y:  11
[ 4 20]
gt:  [ 4 20] x:  0 y:  20
[ 1 15]
gt:  [ 1 15] x:  6 y:  8
[1 9]
gt:  [1 9] x:  3 y:  3
[ 0 24]
gt:  [ 0 24] x:  0 y:  17
[1 7]
gt:  [1 7] x:  6 y:  1
[ 2 20]
gt:  [ 2 20] x:  5 y:  15
[ 3 19]
gt:  [ 3 19] x:  0 y:  17
[ 1 14]
gt:  [ 1 14] x:  1 y:  12
[ 3 21]
gt:  [ 3 21] x:  0 y:  21
[1 5]
gt:  [1 5] x:  6 y:  8
[3 9]
gt:  [3 9] x:  6 y:  9
[1 0]
gt:  [1 0] x:  6 y:  1
[ 1 21]
gt:  [ 1 21] x:  0 y:  18
[ 5 14]
gt:  [ 5 14] x:  5 y:  11
[ 3 21]
gt:  [ 3 21] x:  0 y:  20
[5 0]
gt:  [5 0] x:  1 y:  1
[ 1 24]
gt:  [ 1 24] x:  0 y:  21
[ 0 11]
gt:  [ 0 11] x:  0 y:  17
[3 7]
gt:  [3 7] x:  5 y:  3
[ 5 11]
gt:  [ 5 11] x:  1 y:  1
[ 0 17]
gt:  [ 0 17] x:  5 y:  10
[ 4 21]
gt:  [ 4 21] x:  4 y:  19
[ 1 11]
gt:  [ 1 11] x:  6 y: 

0.023323615160349854

In [36]:
e = localization_error(dataset.grid_labels_test, np.vstack((np.argmax(probx, axis=1), np.argmax(proby, axis=1))).transpose())


In [37]:
e.shape

(343,)

In [38]:
np.min(e)

0.0

In [39]:
np.max(e)

19.646882704388499

In [40]:
np.mean(e)

5.8863938526680331

In [42]:
px.shape

(1, 7)

In [43]:
probx.shape

(343, 7)

In [50]:
predx = np.argmax(probx, axis=1)

In [51]:
predy = np.argmax(proby, axis=1)

In [54]:
pred = np.hstack((probx,proby))

In [53]:
pred

(343, 32)

In [55]:
predy.shape

(343,)

In [56]:
predx.shape

(343,)

In [57]:
pred.shape

(343, 32)

In [65]:
pred =np.vstack((predx,predy)).transpose()

In [66]:
pred.shape

(343, 2)

In [67]:
pred

array([[ 0, 18],
       [ 6, 11],
       [ 6, 19],
       [ 6,  1],
       [ 5, 11],
       [ 0, 18],
       [ 5, 19],
       [ 5, 11],
       [ 0, 20],
       [ 6,  8],
       [ 3,  3],
       [ 0, 17],
       [ 6,  1],
       [ 5, 15],
       [ 0, 17],
       [ 1, 12],
       [ 0, 21],
       [ 6,  8],
       [ 6,  9],
       [ 6,  1],
       [ 0, 18],
       [ 5, 11],
       [ 0, 20],
       [ 1,  1],
       [ 0, 21],
       [ 0, 17],
       [ 5,  3],
       [ 1,  1],
       [ 5, 10],
       [ 4, 19],
       [ 6,  1],
       [ 4, 15],
       [ 6,  1],
       [ 0, 18],
       [ 5, 19],
       [ 5, 19],
       [ 1, 12],
       [ 5, 12],
       [ 0, 17],
       [ 1, 12],
       [ 0, 24],
       [ 6,  3],
       [ 6,  8],
       [ 6,  1],
       [ 1, 12],
       [ 6,  8],
       [ 1, 12],
       [ 1, 12],
       [ 1, 12],
       [ 0, 17],
       [ 6,  8],
       [ 6,  1],
       [ 0, 21],
       [ 6,  9],
       [ 6,  9],
       [ 0, 19],
       [ 5, 11],
       [ 0, 17],
       [ 1, 12

In [69]:
pred[0,:]

array([ 0, 18])

In [71]:
sklearn.metrics.precision_score(dataset.grid_labels_test[:,0], predx)

ValueError: Target is multiclass but average='binary'. Please choose another average setting.

In [72]:
accx

0.30612244897959184

In [73]:
accy

0.087463556851311949

In [74]:
predx

array([0, 6, 6, 6, 5, 0, 5, 5, 0, 6, 3, 0, 6, 5, 0, 1, 0, 6, 6, 6, 0, 5, 0,
       1, 0, 0, 5, 1, 5, 4, 6, 4, 6, 0, 5, 5, 1, 5, 0, 1, 0, 6, 6, 6, 1, 6,
       1, 1, 1, 0, 6, 6, 0, 6, 6, 0, 5, 0, 1, 6, 5, 6, 6, 1, 0, 6, 0, 5, 0,
       5, 5, 0, 2, 3, 5, 5, 6, 6, 0, 3, 0, 5, 0, 1, 4, 6, 5, 0, 1, 6, 0, 1,
       5, 6, 0, 5, 5, 0, 5, 0, 5, 0, 6, 1, 5, 6, 6, 0, 0, 0, 3, 6, 3, 6, 3,
       0, 0, 0, 0, 1, 1, 5, 6, 5, 0, 0, 3, 4, 3, 5, 3, 1, 6, 0, 5, 0, 5, 6,
       0, 5, 6, 5, 6, 5, 6, 6, 3, 0, 1, 5, 5, 0, 1, 5, 6, 0, 6, 4, 6, 6, 6,
       0, 5, 5, 6, 0, 5, 0, 6, 6, 0, 6, 6, 0, 5, 3, 5, 0, 5, 5, 6, 1, 6, 6,
       1, 1, 0, 3, 4, 4, 0, 6, 0, 6, 5, 1, 5, 5, 5, 0, 5, 0, 6, 5, 6, 6, 2,
       2, 6, 0, 6, 1, 5, 0, 0, 0, 1, 0, 6, 3, 1, 0, 0, 1, 5, 6, 3, 6, 3, 6,
       0, 0, 5, 6, 0, 3, 5, 5, 5, 1, 3, 6, 0, 1, 0, 1, 5, 6, 5, 6, 6, 1, 1,
       6, 0, 6, 6, 0, 1, 0, 6, 6, 1, 6, 0, 5, 6, 6, 6, 5, 6, 6, 6, 5, 6, 5,
       0, 0, 3, 0, 0, 5, 0, 1, 5, 5, 6, 1, 0, 6, 5, 6, 5, 5, 5, 6, 0, 1, 6,
       6, 6,

In [76]:
sklearn.metrics.precision_score(dataset.grid_labels_test[:,0], predx)

ValueError: Target is multiclass but average='binary'. Please choose another average setting.

In [78]:
predx.shape

(343,)

In [79]:
sklearn.metrics.precision_score(dataset.grid_labels_test[:,0], predx, average="samples")

ValueError: Sample-based precision, recall, fscore is not meaningful outside multilabel classification. See the accuracy_score instead.

In [80]:
sklearn.metrics.precision_score(dataset.grid_labels_test[:,0], predx, average=None)

array([ 0.36666667,  0.43902439,  0.        ,  0.19047619,  0.4       ,
        0.32051282,  0.21212121])

In [81]:
np.mean(sklearn.metrics.precision_score(dataset.grid_labels_test[:,0], predx, average=None))

0.27554304000297031

In [82]:
sklearn.metrics.precision_score(dataset.grid_labels_test[:,1], predy, average=None)

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array([ 0.14285714,  0.09375   ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.18518519,
        0.25      ,  0.03703704,  0.06896552,  0.        ,  0.        ,
        0.16666667,  0.        ,  0.03125   ,  0.        ,  0.05555556,
        0.25      ,  0.22727273,  0.        ,  0.        ,  0.25      ])

In [83]:
np.mean(sklearn.metrics.precision_score(dataset.grid_labels_test[:,0], predx, average=None))

0.27554304000297031

In [85]:
np.mean(sklearn.metrics.precision_score(dataset.grid_labels_test[:,1], predy, average=None))

0.11034159327262776

In [86]:
accx

0.30612244897959184

In [87]:
accy

0.087463556851311949

In [88]:
np.mean(sklearn.metrics.recall_score(dataset.grid_labels_test[:,0], predx, average=None))

0.28895126117725672

In [89]:
np.mean(sklearn.metrics.precision_score(dataset.grid_labels_test[:,1], predy, average=None))

0.11034159327262776

In [90]:
np.mean(sklearn.metrics.recall_score(dataset.grid_labels_test[:,1], predy, average=None))

0.087074384679647829

In [91]:
s

8

In [92]:
bins, counts, f = cdf(e, 100)
bins = bins[1:]
min_id = np.argmin(np.abs(f - 0.9))
print bins[min_id], f[min_id]
plt.plot(bins, f)
plt.grid(b=True, which='minor', color='r', linestyle='-', alpha=0.2)
plt.minorticks_on()
plt.show()

11.7881296226 0.897959183673


SyntaxError: invalid syntax (<ipython-input-93-392288dd6f82>, line 1)

In [94]:
e_joint = np.array([13.2510264318575	14.9893681679488	16.1700593400152	12.9480748772734	12.9964186853027	11.6165561128130	9.87258804675318	10.0004276456578	12.0394612489648	10.6306923953156	11.2714270288609	10.7340632190587	6.85591114097756	10.0191172717885	10.3405251059876	10.8080480160988	10.6083556108702	7.47933356776918	8.18031258896091	9.64711466984599	9.22761097835699	9.25554368318021	9.00937824826859	11.1635786837236	9.34395564239751	10.6467836915683	10.4073725585048	8.23166005925242	12.6604600565866	9.09793765499117	10.4828759110095	8.15994200596270	8.49035586387743	7.77519329363293	8.82833553003932	10.7119046507239	5.62399583330777	7.40199912978779	6.17294627533988	7.35404215037471	4.89478889220396	7.68869613594711	5.67128789729284	4.50536291211913	np.nan	5.69897021785755	8.29849834815231	4.90482875831842	9.95068865005027	4.34914970577455	4.23738140140945	6.17756373335149	6.93414737687737	5.52934571750844	8.48513277641297	4.86795202431925	2.75631144167450	7.12131442379981	4.15631533754784	9.08810719195538	3.23639179564295	3.03606159968154	2.92846062165080	4.32456957388232	1.50537052759014	2.44062095019210	3.58907712039995	6.16097120369510	5.22983652109448	3.54352146418285	2.83230024604428	5.24028171471484	3.49326242760503	3.30663905085163	4.37529682174252	2.07748798082329	2.60213344659466	5.95379650117632	3.99552205883661	1.04291165033661	2.94104672619242	0.318738375867059	2.09092416516711	2.78288289587020	4.29245437856404	2.47884301015422	3.73543668609265	1.18505721418386	3.99363354243631	2.88028592952713	1.09773487744587	9.00473329609362	1.41974436054764	1.25034656439547	4.90234786107604	6.90841642984103	0.710663474357340	4.65594119164751	4.28531493279498	2.49364844176046	9.56336729741073	3.24325186417288	1.27060107097494	3.59494914838388	7.22478563578355	5.86139192888617	5.82757326962160	3.33740896917291	3.21647008812653	1.55998079998170	5.55153637988186	4.64317176039482	11.3056904798819	8.93083609290333	np.nan	4.43834515896954	4.06636872321896	1.91931005426642	5.24666370048571	5.55177747751193	10.5299585830915	4.11069400884111	5.91995597368467	4.39990197544386	3.50117302120080	7.45404757591254	8.33294848287958	5.34386122045249	11.8964883079406	7.15577882871285	5.51956344249666	7.99142890980480	5.93080624425842	8.45232365511315	7.49541591341740	5.16407686573824	7.39975386802367	4.87279816108130	9.33864183809120	4.75535097587758	7.49089812502274	8.69958680211042	6.97551944338098	8.08736150236295	6.94207586250742	7.00692446075230	6.82670877775403	8.67032258505448	7.44830063169332	6.68941502295000	9.63282003405974	8.19286369187143	10.9052085983262	5.30639233538977	9.56313298140363	10.3967819160835	np.nan	np.nan	np.nan	11.7114645667025	12.7248604831159	11.0557967403117	11.7383882459974	np.nan	np.nan	np.nan	9.77383796390541	12.0251602593006	13.6092815217801	12.6019787027883	np.nan	np.nan	np.nan	7.93158273338898	10.7360217778744])

SyntaxError: invalid syntax (<ipython-input-94-98fc0d799f14>, line 1)

In [95]:
e_joint = np.array([13.2510264318575, 14.9893681679488, 16.1700593400152, 12.9480748772734, 12.9964186853027, 11.6165561128130, 9.87258804675318, 10.0004276456578, 12.0394612489648, 10.6306923953156, 11.2714270288609, 10.7340632190587, 6.85591114097756, 10.0191172717885, 10.3405251059876, 10.8080480160988, 10.6083556108702, 7.47933356776918, 8.18031258896091, 9.64711466984599, 9.22761097835699, 9.25554368318021, 9.00937824826859, 11.1635786837236, 9.34395564239751, 10.6467836915683, 10.4073725585048, 8.23166005925242, 12.6604600565866, 9.09793765499117, 10.4828759110095, 8.15994200596270, 8.49035586387743, 7.77519329363293, 8.82833553003932, 10.7119046507239, 5.62399583330777, 7.40199912978779, 6.17294627533988, 7.35404215037471, 4.89478889220396, 7.68869613594711, 5.67128789729284, 4.50536291211913, np.nan, 5.69897021785755, 8.29849834815231, 4.90482875831842, 9.95068865005027, 4.34914970577455, 4.23738140140945, 6.17756373335149, 6.93414737687737, 5.52934571750844, 8.48513277641297, 4.86795202431925, 2.75631144167450, 7.12131442379981, 4.15631533754784, 9.08810719195538, 3.23639179564295, 3.03606159968154, 2.92846062165080, 4.32456957388232, 1.50537052759014, 2.44062095019210, 3.58907712039995, 6.16097120369510, 5.22983652109448, 3.54352146418285, 2.83230024604428, 5.24028171471484, 3.49326242760503, 3.30663905085163, 4.37529682174252, 2.07748798082329, 2.60213344659466, 5.95379650117632, 3.99552205883661, 1.04291165033661, 2.94104672619242, 0.318738375867059, 2.09092416516711, 2.78288289587020, 4.29245437856404, 2.47884301015422, 3.73543668609265, 1.18505721418386, 3.99363354243631, 2.88028592952713, 1.09773487744587, 9.00473329609362, 1.41974436054764, 1.25034656439547, 4.90234786107604, 6.90841642984103, 0.710663474357340, 4.65594119164751, 4.28531493279498, 2.49364844176046, 9.56336729741073, 3.24325186417288, 1.27060107097494, 3.59494914838388, 7.22478563578355, 5.86139192888617, 5.82757326962160, 3.33740896917291, 3.21647008812653, 1.55998079998170, 5.55153637988186, 4.64317176039482, 11.3056904798819, 8.93083609290333, np.nan, 4.43834515896954, 4.06636872321896, 1.91931005426642, 5.24666370048571, 5.55177747751193, 10.5299585830915, 4.11069400884111, 5.91995597368467, 4.39990197544386, 3.50117302120080, 7.45404757591254, 8.33294848287958, 5.34386122045249, 11.8964883079406, 7.15577882871285, 5.51956344249666, 7.99142890980480, 5.93080624425842, 8.45232365511315, 7.49541591341740, 5.16407686573824, 7.39975386802367, 4.87279816108130, 9.33864183809120, 4.75535097587758, 7.49089812502274, 8.69958680211042, 6.97551944338098, 8.08736150236295, 6.94207586250742, 7.00692446075230, 6.82670877775403, 8.67032258505448, 7.44830063169332, 6.68941502295000, 9.63282003405974, 8.19286369187143, 10.9052085983262, 5.30639233538977, 9.56313298140363, 10.3967819160835, np.nan, np.nan, np.nan, 11.7114645667025, 12.7248604831159, 11.0557967403117, 11.7383882459974, np.nan, np.nan, np.nan, 9.77383796390541, 12.0251602593006, 13.6092815217801, 12.6019787027883, np.nan, np.nan, np.nan, 7.93158273338898, 10.7360217778744])

In [96]:
e_joint.shape

(175,)

In [98]:
np.nanmean(e_joint)

6.8586855924653172

In [99]:
e_ind = np.array([12.3727617133558, 12.1710710605764, 12.4011973077025, 12.4347778026124, 11.3783622838354, 11.5384983868465, 11.3450626605196, 10.0976289691737, 12.8102434533037, 10.3746327362212, 9.71529532328172, 11.4940650910189, 8.08233119101387, 10.3812705244149, 14.5378028515914, 9.57829687981682, 9.44318274209461, 8.25465031865468, 9.15672331423657, 9.05871805165576, 9.94533504915979, 9.42251070236942, 7.62889882742005, 8.69285708719568, 10.0552558920969, 11.3539038136918, 12.0144278059953, 10.4786295105931, 10.3716433970594, 13.2836083767022, 9.90256088767992, 8.88238489251153, 9.40522931479187, 7.99170322327927, 8.37428664841259, 6.79606254486408, 7.77083276635543, 8.24625971987557, 6.20488002608978, 8.00707328041077, 4.91753439539160, 7.68712666534347, 4.95377079960326, 6.06295283028496, np.nan, 5.70944010301909, 4.27050035111508, 6.66622424828515, 7.93561811189016, 7.51091577476927, 4.51045358087360, 4.81015104833690, 5.80473321513260, 5.74908070202269, 4.36986602981687, 4.05743538190188, 3.67935692921131, 8.53341863643902, 4.77770581839723, 3.28815154920384, 2.88821597690459, 9.16352336972637, 2.64730287783439, 3.17598137520200, 4.79952339339642, 8.20765794604120, 3.89596630365999, 7.83358836448300, 4.39841442336072, 3.94814952369843, 2.96871727548811, 1.91932673594179, 1.19999555807042, 2.80173141530885, 4.00029098940447, 2.22463315578409, 2.46075602480804, 3.70345814590143, 2.71771973429521, 0.740490904287222, 1.98760366521700, 1.84394605407122, 1.94379931364180, 4.95695648460087, 6.22817956125622, 1.88537367354447, 1.60125084759578, 0.541183887414137, 3.00036723447178, 0.495657564464137, 2.23913168961711, 6.54341345711130, 1.84050546791375, 1.68234251784020, 2.64050726860666, 7.93490644632601, 1.81245142069700, 4.94881343225285, 7.73842085981395, 2.91172680328644, 2.41495113747021, 2.25657206079366, 1.99363736053425, 1.37095538866732, 4.25915998620609, 6.23676610455086, 6.43009095289363, 2.77982399770137, 3.75261439878457, 2.70613929435420, 4.32388842016973, 5.40488838432438, 6.62971250960876, 5.79176382746723, np.nan, 5.37069442428371, 4.61796022981518, 5.23479353862896, 2.83773038878489, 5.57517306573291, 5.85645669679685, 4.59109398110889, 4.28518046457428, 2.80432281072965, 4.00597343075254, 8.03403915824452, 8.56364356929840, 5.80721325499007, 6.06861766320159, 6.08976063343921, 8.11250071270014, 9.68993455627306, 6.18850438713723, 10.8624276838845, 7.02662657399424, 5.09548823659970, 7.40784616482525, 4.96010171496548, 9.13642370966284, 7.69393451486428, 7.80386926581822, 6.65554501920042, 6.93347862103469, 8.33286371231757, 4.64108165797026, 7.84112403141941, 6.16449467550636, 8.66982433158829, 7.60415664031039, 7.40830922484374, 7.08352417219826, 6.38612301891201, 7.01557795818938, 7.94154423764527, 9.41634259973889, 9.77883625136047, np.nan, np.nan, np.nan, 7.58741064013001, 12.2093100050579, 11.1054749488758, 10.9657686974282, np.nan, np.nan, np.nan, 11.3693262414981, 12.8206659554575, 13.5433655588394, 12.2066874899715, np.nan, np.nan, np.nan, 8.12455937842932, 11.8241673338695])

In [100]:
e_ind.shape

(175,)

In [101]:
b_bayes, c_bayes, f_bayes= cdf(e, 100)
b_bayes = b_bayes[1:]
min_id_bayes = np.argmin(np.abs(f_bayes - 0.9))
print b_bayes[min_id_bayes], f_bayes[min_id_bayes]
plt.plot(bins, f)
plt.grid(b=True, which='minor', color='r', linestyle='-', alpha=0.2)
plt.minorticks_on()
plt.show()

11.7881296226 0.897959183673


In [102]:
b_bayes, c_bayes, f_bayes= cdf(e, 100)
b_bayes = b_bayes[1:]
min_id_bayes = np.argmin(np.abs(f_bayes - 0.9))
b_joint, c_joint, f_joint= cdf(e_joint, 100)
b_joint = b_joint[1:]
min_id_joint = np.argmin(np.abs(f_joint - 0.9))
b_ind, c_ind, f_ind= cdf(e_ind, 100)
b_ind = b_ind[1:]
min_id_ind = np.argmin(np.abs(f_ind - 0.9))
print b_bayes[min_id_bayes], f_bayes[min_id_bayes], b_joint[min_id_joint], f_joint[min_id_joint], b_ind[min_id_ind], f_ind[min_id_ind]
plt.plot(b_bayes, f_bayes, 'r', b_joint, f_joint, 'k', b_ind, f_ind, 'b')
plt.grid(b=True, which='minor', color='r', linestyle='-', alpha=0.2)
plt.minorticks_on()
plt.show()

ValueError: range parameter must be finite.

In [103]:
e_joint = np.array([13.2510264318575, 14.9893681679488, 16.1700593400152, 12.9480748772734, 12.9964186853027, 11.6165561128130, 9.87258804675318, 10.0004276456578, 12.0394612489648, 10.6306923953156, 11.2714270288609, 10.7340632190587, 6.85591114097756, 10.0191172717885, 10.3405251059876, 10.8080480160988, 10.6083556108702, 7.47933356776918, 8.18031258896091, 9.64711466984599, 9.22761097835699, 9.25554368318021, 9.00937824826859, 11.1635786837236, 9.34395564239751, 10.6467836915683, 10.4073725585048, 8.23166005925242, 12.6604600565866, 9.09793765499117, 10.4828759110095, 8.15994200596270, 8.49035586387743, 7.77519329363293, 8.82833553003932, 10.7119046507239, 5.62399583330777, 7.40199912978779, 6.17294627533988, 7.35404215037471, 4.89478889220396, 7.68869613594711, 5.67128789729284, 4.50536291211913, 5.69897021785755, 8.29849834815231, 4.90482875831842, 9.95068865005027, 4.34914970577455, 4.23738140140945, 6.17756373335149, 6.93414737687737, 5.52934571750844, 8.48513277641297, 4.86795202431925, 2.75631144167450, 7.12131442379981, 4.15631533754784, 9.08810719195538, 3.23639179564295, 3.03606159968154, 2.92846062165080, 4.32456957388232, 1.50537052759014, 2.44062095019210, 3.58907712039995, 6.16097120369510, 5.22983652109448, 3.54352146418285, 2.83230024604428, 5.24028171471484, 3.49326242760503, 3.30663905085163, 4.37529682174252, 2.07748798082329, 2.60213344659466, 5.95379650117632, 3.99552205883661, 1.04291165033661, 2.94104672619242, 0.318738375867059, 2.09092416516711, 2.78288289587020, 4.29245437856404, 2.47884301015422, 3.73543668609265, 1.18505721418386, 3.99363354243631, 2.88028592952713, 1.09773487744587, 9.00473329609362, 1.41974436054764, 1.25034656439547, 4.90234786107604, 6.90841642984103, 0.710663474357340, 4.65594119164751, 4.28531493279498, 2.49364844176046, 9.56336729741073, 3.24325186417288, 1.27060107097494, 3.59494914838388, 7.22478563578355, 5.86139192888617, 5.82757326962160, 3.33740896917291, 3.21647008812653, 1.55998079998170, 5.55153637988186, 4.64317176039482, 11.3056904798819, 8.93083609290333, 4.43834515896954, 4.06636872321896, 1.91931005426642, 5.24666370048571, 5.55177747751193, 10.5299585830915, 4.11069400884111, 5.91995597368467, 4.39990197544386, 3.50117302120080, 7.45404757591254, 8.33294848287958, 5.34386122045249, 11.8964883079406, 7.15577882871285, 5.51956344249666, 7.99142890980480, 5.93080624425842, 8.45232365511315, 7.49541591341740, 5.16407686573824, 7.39975386802367, 4.87279816108130, 9.33864183809120, 4.75535097587758, 7.49089812502274, 8.69958680211042, 6.97551944338098, 8.08736150236295, 6.94207586250742, 7.00692446075230, 6.82670877775403, 8.67032258505448, 7.44830063169332, 6.68941502295000, 9.63282003405974, 8.19286369187143, 10.9052085983262, 5.30639233538977, 9.56313298140363, 10.3967819160835, 11.7114645667025, 12.7248604831159, 11.0557967403117, 11.7383882459974, 9.77383796390541, 12.0251602593006, 13.6092815217801, 12.6019787027883, 7.93158273338898, 10.7360217778744])

In [104]:
e_ind = np.array([12.3727617133558, 12.1710710605764, 12.4011973077025, 12.4347778026124, 11.3783622838354, 11.5384983868465, 11.3450626605196, 10.0976289691737, 12.8102434533037, 10.3746327362212, 9.71529532328172, 11.4940650910189, 8.08233119101387, 10.3812705244149, 14.5378028515914, 9.57829687981682, 9.44318274209461, 8.25465031865468, 9.15672331423657, 9.05871805165576, 9.94533504915979, 9.42251070236942, 7.62889882742005, 8.69285708719568, 10.0552558920969, 11.3539038136918, 12.0144278059953, 10.4786295105931, 10.3716433970594, 13.2836083767022, 9.90256088767992, 8.88238489251153, 9.40522931479187, 7.99170322327927, 8.37428664841259, 6.79606254486408, 7.77083276635543, 8.24625971987557, 6.20488002608978, 8.00707328041077, 4.91753439539160, 7.68712666534347, 4.95377079960326, 6.06295283028496, 5.70944010301909, 4.27050035111508, 6.66622424828515, 7.93561811189016, 7.51091577476927, 4.51045358087360, 4.81015104833690, 5.80473321513260, 5.74908070202269, 4.36986602981687, 4.05743538190188, 3.67935692921131, 8.53341863643902, 4.77770581839723, 3.28815154920384, 2.88821597690459, 9.16352336972637, 2.64730287783439, 3.17598137520200, 4.79952339339642, 8.20765794604120, 3.89596630365999, 7.83358836448300, 4.39841442336072, 3.94814952369843, 2.96871727548811, 1.91932673594179, 1.19999555807042, 2.80173141530885, 4.00029098940447, 2.22463315578409, 2.46075602480804, 3.70345814590143, 2.71771973429521, 0.740490904287222, 1.98760366521700, 1.84394605407122, 1.94379931364180, 4.95695648460087, 6.22817956125622, 1.88537367354447, 1.60125084759578, 0.541183887414137, 3.00036723447178, 0.495657564464137, 2.23913168961711, 6.54341345711130, 1.84050546791375, 1.68234251784020, 2.64050726860666, 7.93490644632601, 1.81245142069700, 4.94881343225285, 7.73842085981395, 2.91172680328644, 2.41495113747021, 2.25657206079366, 1.99363736053425, 1.37095538866732, 4.25915998620609, 6.23676610455086, 6.43009095289363, 2.77982399770137, 3.75261439878457, 2.70613929435420, 4.32388842016973, 5.40488838432438, 6.62971250960876, 5.79176382746723, 5.37069442428371, 4.61796022981518, 5.23479353862896, 2.83773038878489, 5.57517306573291, 5.85645669679685, 4.59109398110889, 4.28518046457428, 2.80432281072965, 4.00597343075254, 8.03403915824452, 8.56364356929840, 5.80721325499007, 6.06861766320159, 6.08976063343921, 8.11250071270014, 9.68993455627306, 6.18850438713723, 10.8624276838845, 7.02662657399424, 5.09548823659970, 7.40784616482525, 4.96010171496548, 9.13642370966284, 7.69393451486428, 7.80386926581822, 6.65554501920042, 6.93347862103469, 8.33286371231757, 4.64108165797026, 7.84112403141941, 6.16449467550636, 8.66982433158829, 7.60415664031039, 7.40830922484374, 7.08352417219826, 6.38612301891201, 7.01557795818938, 7.94154423764527, 9.41634259973889, 9.77883625136047, 7.58741064013001, 12.2093100050579, 11.1054749488758, 10.9657686974282, 11.3693262414981, 12.8206659554575, 13.5433655588394, 12.2066874899715, 8.12455937842932, 11.8241673338695])

In [105]:
b_bayes, c_bayes, f_bayes= cdf(e, 100)
b_bayes = b_bayes[1:]
min_id_bayes = np.argmin(np.abs(f_bayes - 0.9))
b_joint, c_joint, f_joint= cdf(e_joint, 100)
b_joint = b_joint[1:]
min_id_joint = np.argmin(np.abs(f_joint - 0.9))
b_ind, c_ind, f_ind= cdf(e_ind, 100)
b_ind = b_ind[1:]
min_id_ind = np.argmin(np.abs(f_ind - 0.9))
print b_bayes[min_id_bayes], f_bayes[min_id_bayes], b_joint[min_id_joint], f_joint[min_id_joint], b_ind[min_id_ind], f_ind[min_id_ind]
plt.plot(b_bayes, f_bayes, 'r', b_joint, f_joint, 'k', b_ind, f_ind, 'b')
plt.grid(b=True, which='minor', color='r', linestyle='-', alpha=0.2)
plt.minorticks_on()
plt.show()

11.7881296226 0.897959183673 11.2561498411 0.896341463415 11.4485308884 0.908536585366


In [106]:
b_bayes, c_bayes, f_bayes= cdf(e, 100)
b_bayes = b_bayes[1:]
min_id_bayes = np.argmin(np.abs(f_bayes - 0.9))
b_joint, c_joint, f_joint= cdf(e_joint, 100)
b_joint = b_joint[1:]
min_id_joint = np.argmin(np.abs(f_joint - 0.9))
b_ind, c_ind, f_ind= cdf(e_ind, 100)
b_ind = b_ind[1:]
min_id_ind = np.argmin(np.abs(f_ind - 0.9))
print b_bayes[min_id_bayes], f_bayes[min_id_bayes], b_joint[min_id_joint], f_joint[min_id_joint], b_ind[min_id_ind], f_ind[min_id_ind]
plt.plot(b_bayes, f_bayes, 'r', b_joint, f_joint, 'k', b_ind, f_ind, 'b')
plt.grid(b=True, which='minor', color='r', linestyle='-', alpha=0.2)
plt.minorticks_on()
plt.show()

11.7881296226 0.897959183673 11.2561498411 0.896341463415 11.4485308884 0.908536585366


In [107]:
b_bayes, c_bayes, f_bayes= cdf(e, 100)
b_bayes = b_bayes[1:]
min_id_bayes = np.argmin(np.abs(f_bayes - 0.9))
b_joint, c_joint, f_joint= cdf(e_joint, 100)
b_joint = b_joint[1:]
min_id_joint = np.argmin(np.abs(f_joint - 0.9))
b_ind, c_ind, f_ind= cdf(e_ind, 100)
b_ind = b_ind[1:]
min_id_ind = np.argmin(np.abs(f_ind - 0.9))
print b_bayes[min_id_bayes], f_bayes[min_id_bayes], b_joint[min_id_joint], f_joint[min_id_joint], b_ind[min_id_ind], f_ind[min_id_ind]
plt.plot(b_bayes, f_bayes, 'r', label="Naive Bayes")
plt.plot(b_joint, f_joint, 'k', label="LDPL (Joint Estimation)")b_ind, f_ind, 'b')
plt.grid(b=True, which='minor', color='r', linestyle='-', alpha=0.2)

plt.minorticks_on()
plt.show()

SyntaxError: invalid syntax (<ipython-input-107-9eb798fb1b9b>, line 12)

In [108]:
b_bayes, c_bayes, f_bayes= cdf(e, 100)
b_bayes = b_bayes[1:]
min_id_bayes = np.argmin(np.abs(f_bayes - 0.9))
b_joint, c_joint, f_joint= cdf(e_joint, 100)
b_joint = b_joint[1:]
min_id_joint = np.argmin(np.abs(f_joint - 0.9))
b_ind, c_ind, f_ind= cdf(e_ind, 100)
b_ind = b_ind[1:]
min_id_ind = np.argmin(np.abs(f_ind - 0.9))
print b_bayes[min_id_bayes], f_bayes[min_id_bayes], b_joint[min_id_joint], f_joint[min_id_joint], b_ind[min_id_ind], f_ind[min_id_ind]
plt.plot(b_bayes, f_bayes, 'r', label="Naive Bayes")
plt.plot(b_joint, f_joint, 'k', label="LDPL (Joint Estimation)")
plt.plot(b_ind, f_ind, 'b', label="LDPL (Ind. Estimation)")
plt.grid(b=True, which='minor', color='r', linestyle='-', alpha=0.2)

plt.minorticks_on()
plt.show()

11.7881296226 0.897959183673 11.2561498411 0.896341463415 11.4485308884 0.908536585366


In [109]:
b_bayes, c_bayes, f_bayes= cdf(e, 100)
b_bayes = b_bayes[1:]
min_id_bayes = np.argmin(np.abs(f_bayes - 0.9))
b_joint, c_joint, f_joint= cdf(e_joint, 100)
b_joint = b_joint[1:]
min_id_joint = np.argmin(np.abs(f_joint - 0.9))
b_ind, c_ind, f_ind= cdf(e_ind, 100)
b_ind = b_ind[1:]
min_id_ind = np.argmin(np.abs(f_ind - 0.9))
print b_bayes[min_id_bayes], f_bayes[min_id_bayes], b_joint[min_id_joint], f_joint[min_id_joint], b_ind[min_id_ind], f_ind[min_id_ind]
plt.plot(b_bayes, f_bayes, 'r', label="Naive Bayes")
plt.plot(b_joint, f_joint, 'k', label="LDPL (Joint Estimation)")
plt.plot(b_ind, f_ind, 'b', label="LDPL (Ind. Estimation)")
plt.grid(b=True, which='minor', color='r', linestyle='-', alpha=0.2)
plt.legend()
plt.minorticks_on()
plt.show()

11.7881296226 0.897959183673 11.2561498411 0.896341463415 11.4485308884 0.908536585366


In [111]:
b_bayes, c_bayes, f_bayes= cdf(e, 100)
b_bayes = b_bayes[1:]
min_id_bayes = np.argmin(np.abs(f_bayes - 0.9))
b_joint, c_joint, f_joint= cdf(e_joint, 100)
b_joint = b_joint[1:]
min_id_joint = np.argmin(np.abs(f_joint - 0.9))
b_ind, c_ind, f_ind= cdf(e_ind, 100)
b_ind = b_ind[1:]
min_id_ind = np.argmin(np.abs(f_ind - 0.9))
print b_bayes[min_id_bayes], f_bayes[min_id_bayes], b_joint[min_id_joint], f_joint[min_id_joint], b_ind[min_id_ind], f_ind[min_id_ind]
plt.plot(b_bayes, f_bayes, 'r', label="Naive Bayes")
plt.plot(b_joint, f_joint, 'k', label="LDPL (Joint Estimation)")
plt.plot(b_ind, f_ind, 'b', label="LDPL (Ind. Estimation)")
plt.grid(b=True, which='minor', color='r', linestyle='-', alpha=0.2)
plt.legend(loc=4)
plt.ylabel("CDF of Localization Error")
plt.minorticks_on()
plt.show()

11.7881296226 0.897959183673 11.2561498411 0.896341463415 11.4485308884 0.908536585366


In [113]:
b_bayes, c_bayes, f_bayes= cdf(e, 100)
b_bayes = b_bayes[1:]
min_id_bayes = np.argmin(np.abs(f_bayes - 0.9))
b_joint, c_joint, f_joint= cdf(e_joint, 100)
b_joint = b_joint[1:]
min_id_joint = np.argmin(np.abs(f_joint - 0.9))
b_ind, c_ind, f_ind= cdf(e_ind, 100)
b_ind = b_ind[1:]
min_id_ind = np.argmin(np.abs(f_ind - 0.9))
print b_bayes[min_id_bayes], f_bayes[min_id_bayes], b_joint[min_id_joint], f_joint[min_id_joint], b_ind[min_id_ind], f_ind[min_id_ind]
plt.plot(b_bayes, f_bayes, 'r', label="Naive Bayes")
plt.plot(b_joint, f_joint, 'k', label="LDPL (Joint Estimation)")
plt.plot(b_ind, f_ind, 'b', label="LDPL (Ind. Estimation)")
plt.grid(b=True, which='minor', color='r', linestyle='-', alpha=0.2)
plt.legend(loc=4)
plt.ylabel("CDF of Localization Error")
plt.xlabel("Error (Euclidian Distance)")
plt.minorticks_on()
plt.show()

11.7881296226 0.897959183673 11.2561498411 0.896341463415 11.4485308884 0.908536585366


In [123]:
plt.bar(b_bayes, c_bayes, 'r')
plt.bar(b_joint, c_joint, 'k')
plt.bar(b_ind, c_ind, 'b')
plt.grid(b=True, which='minor', color='r', linestyle='-', alpha=0.2)
plt.legend(loc=4)
plt.ylabel("Histogram of Localization Error")
plt.xlabel("Error (Euclidian Distance)")
plt.minorticks_on()
plt.show()

TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('S32') dtype('S32') dtype('S32')

In [116]:
type(c_bayes)

numpy.ndarray

In [118]:
c_bayes.type()

AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [119]:
np.dtype(c_bayes)

TypeError: data type not understood

In [121]:
c_bayes.dtype

dtype('int64')

In [122]:
b_bayes.dtype

dtype('float64')

In [133]:
plt.bar(b_bayes, c_bayes/343, label="Naive Bayes", color='red')
plt.bar(b_joint, c_joint/164, label="LDPL (Joint Estimation)", color='black')
plt.bar(b_ind, c_ind/164, label="LDPL (Joint Estimation)", color='blue')
plt.grid(b=True, which='minor', color='r', linestyle='-', alpha=0.2)
plt.legend(loc=4)
plt.ylabel("Histogram of Localization Error")
plt.xlabel("Error (Euclidian Distance)")
plt.minorticks_on()
plt.show()

In [132]:
print np.sum(c_bayes/343), np.sum(c_joint/164), np.sum(c_ind/164
                                                      )

1.0 1.0 1.0


In [148]:
ax1 = plt.subplot(3, 1, 1)
n, bins, patches = plt.hist(e, 50, normed=1, facecolor='red', alpha=0.75)
plt.title("Error (Euclidian Distance)")
plt.minorticks_on()
plt.xlabel("Naive Bayes")
plt.grid(b=True, which='minor', color='r', linestyle='-', alpha=0.2)
ax2 = plt.subplot(3, 1, 2, sharex=ax1, sharey=ax1)
n, bins, patches = plt.hist(e_joint, 50, normed=1, facecolor='black', alpha=0.75)
plt.xlabel("LDPL (Joint Estimation)")
plt.grid(b=True, which='minor', color='r', linestyle='-', alpha=0.2)
plt.minorticks_on()

plt.ylabel("Histogram of Localization Error")
plt.subplot(3, 1, 3, sharex=ax1, sharey=ax1)
plt.grid(b=True, which='minor', color='r', linestyle='-', alpha=0.2)
n, bins, patches = plt.hist(e_ind, 50, normed=1, facecolor='blue', alpha=0.75)
plt.xlabel("LDPL (Ind. Estimation)")
plt.minorticks_on()
plt.show()


In [150]:
np.mean(e_joint)

6.8586855924653172

array([ 0.14285714,  0.09375   ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.18518519,
        0.25      ,  0.03703704,  0.06896552,  0.        ,  0.        ,
        0.16666667,  0.        ,  0.03125   ,  0.        ,  0.05555556,
        0.25      ,  0.22727273,  0.        ,  0.        ,  0.25      ])

In [152]:
np.mean(sklearn.metrics.precision_score(dataset.grid_labels_test[:,1], predy, average=None))

0.11034159327262776

In [154]:
sklearn.metrics.average_precision_score(dataset.grid_labels_test[:,1], predy)

ValueError: multiclass format is not supported

In [155]:
py

array([[ 0.03830168],
       [ 0.04666484],
       [ 0.04310316],
       [ 0.04446492],
       [ 0.04035215],
       [ 0.04340791],
       [ 0.03844331],
       [ 0.044554  ],
       [ 0.04881487],
       [ 0.05035322],
       [ 0.05351018],
       [ 0.05818844],
       [ 0.04925768],
       [ 0.04755846],
       [ 0.04425744],
       [ 0.04852   ],
       [ 0.03276276],
       [ 0.03932744],
       [ 0.03108758],
       [ 0.05033346],
       [ 0.02321687],
       [ 0.03007603],
       [ 0.01797583],
       [ 0.02108745],
       [ 0.01438033]])

In [156]:
predy

array([18, 11, 19,  1, 11, 18, 19, 11, 20,  8,  3, 17,  1, 15, 17, 12, 21,
        8,  9,  1, 18, 11, 20,  1, 21, 17,  3,  1, 10, 19,  1, 15,  1, 18,
       19, 19, 12, 12, 17, 12, 24,  3,  8,  1, 12,  8, 12, 12, 12, 17,  8,
        1, 21,  9,  9, 19, 11, 17, 12,  1, 11,  9, 19, 12, 17,  8, 17, 11,
       17, 11, 15, 17,  4,  0, 19, 11,  8,  4, 18,  1, 20, 11, 21, 12, 19,
       11, 11, 21,  1,  8, 18, 11, 21,  4, 19, 11, 19, 17, 19, 19, 19, 18,
        1,  1, 15,  8,  3, 17, 18, 18,  2,  1,  0, 19,  1, 21, 18, 17, 19,
       12, 12, 11, 19, 15, 21, 21,  0, 19,  3,  9,  0,  1,  1, 21, 15, 18,
       19, 17, 19, 11, 19, 15,  8, 19, 18,  8,  3, 17,  1, 11, 19, 18, 12,
       15,  3, 17,  1,  9,  8,  1, 19, 17, 15, 15,  8, 24, 15, 17,  8,  3,
       21,  9,  1, 17, 19,  3, 15, 18, 19, 15,  9, 17, 18,  5, 12, 12, 18,
        3, 19, 19, 17,  9, 21,  9, 19,  1, 11, 15, 11, 18, 19, 18,  8, 15,
        8,  8,  9,  9,  9, 18,  9, 11, 19, 18, 20, 17,  1, 20,  1,  2,  1,
       17, 17, 12, 15,  1

In [157]:
sklearn.metrics.average_precision_score(dataset.grid_labels_test[:,1], py)

ValueError: multiclass format is not supported